In [ ]:
#INSTALANDO O BEAM
#!pip install apache-beam[interactive]

#INSTALANDO OS RECURSOS PARA QUE O BEAM POSSA INTERAGIR COM O CLOUD
!pip install apache-beam[gcp]


In [3]:
import apache_beam as beam
import os #Importando o sistema operacional


In [4]:
#ATRIBUINDO A CHAVE DE SERVIÇO DO GOOGLE CLOUD À UMA VARIÁVEL
serviceAccount = '/content/drive/MyDrive/ingest-ana-304af4810579.json' 

#DANDO AUTORIZACAO AO SISTEMA OPERACIONAL
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount


In [7]:
#CRIANDO UM PIPELINE
p5 = beam.Pipeline()

#DEFININDO OS PASSOS DO PIPELINE
voosComAtrasoTempo = (
    p5 #p3 está recebendo esses passos
    |'Extrair os Dados' >> beam.io.ReadFromText('/content/drive/MyDrive/voos_sample.csv', skip_header_lines=0) #Rótulo do passo que está sendo executado - Passo executa funcao
    |'Separador do Dataframe' >> beam.Map(lambda record: record.split(',')) #Informando o tipo do separador
    |'Aeroportos Atrasados' >> beam.Filter(lambda record: int(record[8]) > 0) #Se maior que zero, houve atraso no aeroporto
    |'Agregacao de Colunas' >> beam.Map(lambda record: (record[3],int(record[8]))) #Apresentará apenas duas colunas
    |'Combinacao de Dados' >> beam.CombinePerKey(sum) #Soma os valores de colunas iguais que foram retornados
)

voosComAtrasoQuantidade = (
    p5 #p3 está recebendo esses passos - quando usado novamente, ele fica armazenado em outro lugar.
    |'1Extrair os Dados' >> beam.io.ReadFromText('/content/drive/MyDrive/voos_sample.csv', skip_header_lines=0) #Rótulo do passo que está sendo executado - Passo executa funcao
    |'1Separador do Dataframe' >> beam.Map(lambda record: record.split(',')) #Informando o tipo do separador
    |'1Aeroportos Atrasados' >> beam.Filter(lambda record: int(record[8]) > 0) #Se maior que zero, houve atraso no aeroporto
    |'1Agregacao de Colunas' >> beam.Map(lambda record: (record[3],int(record[8]))) #Apresentará apenas duas colunas
    |'1Quantidade de Atrasos' >> beam.combiners.Count.PerKey() #Conta quantas vezes tal coluna retornou um valor
)

In [10]:
#RESULTADO FINAL
tabelaFinal = (
    {'atrasoTempo':voosComAtrasoTempo, 'atrasoQuantidade':voosComAtrasoQuantidade} #Passa os pipes como dicionarios
    |'Agrupamento dos Pipelines' >> beam.CoGroupByKey() #Agrupa os resultados das pipelines
    |'Salvando no Stoage' >> beam.io.WriteToText('gs://data-lake-ana/TESTE/teste.csv') #Enviando o resultado para o bucket no GCP
)

#EXECUTANDO O PIPELINE
p5.run()